<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Simple-psychometric-functions" data-toc-modified-id="Simple-psychometric-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Simple psychometric functions</a></span></li><li><span><a href="#Logistic-regression" data-toc-modified-id="Logistic-regression-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Logistic regression</a></span><ul class="toc-item"><li><span><a href="#Single-predictor" data-toc-modified-id="Single-predictor-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Single predictor</a></span></li></ul></li></ul></div>

In [1]:
from IPython.display import HTML
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()">Toggle code cells ON/OFF</a>.''')

In [3]:
plt.rcParams['figure.figsize'] = [14, 7]
plt.rcParams.update({'font.size': 19})

## Simple psychometric functions

Weibull function:
$$F_W(|x|;\alpha_W, \beta_W)=1-\exp\left(-\left(\frac{|x|}{\alpha_W}\right)^{\beta_W}\right), \quad \alpha_W, \beta_W > 0$$

In [4]:
def F_W(x, a, b):
    return 1-np.exp(-((x/a)**b))

In [5]:
def psi(x, l, g, F, a, b):
    return g+(1-g-l)*F(x, a, b)

Corresponding psychometric function (interactive widget below):
$$\psi_\text{corr}(|x|;\theta_\text{corr})=\gamma_\text{corr}+(1-\gamma_\text{corr}-\lambda_\text{corr})F_W(|x|;\alpha_W, \beta_W)$$

In [6]:
plt.rcParams['figure.figsize'] = [10, 7]

In [7]:
@interact
def psi_corr(threshold=(.01, 100, 1), slope=(.01, 20, 1), lapse=(0,.1,.01)):
    g=1/2
    x = np.linspace(0,100,200)
#     fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
    _, ax = plt.subplots()
    ax.plot(x, 100 * psi(x, lapse, g, F_W, threshold, slope), linewidth=3)
    ax.plot([threshold, threshold], [50, 100*psi(threshold, lapse, g, F_W, threshold, slope)], color='green')
    ax.set(xlim=(0, 100), ylim=(50,100),
           xlabel="coherence",
           ylabel="percent correct",
           title="Weibull + lapses")

A Jupyter Widget

Logistic function:
$$\displaystyle
F_L(x;\alpha_L, \beta_L)=\frac{1}{1+\exp\left(-\beta_L(x-\alpha_L)\right)}, \quad \alpha_L\in\mathbb{R}, \beta_L > 0$$

In [8]:
def logistic_base(x, f=None):
    """
    Base function to apply a logistic transformation. 
    
    :param x: values at which logistic function should be evaluated
    :param f: a function. If None, identity is used
    """
    if f is None:
        f = lambda b: b  # f is the identity function
    return 1/(1+np.exp(-f(x)))

In [9]:
def F_L(x, a, b):
    exp_arg = lambda x: b*(x-a)
    return logistic_base(x, exp_arg)

Corresponding psychometric function (interactive widget below):
$$\psi_\text{choice}(x;\theta_\text{choice})=\lambda_\text{choice}+(1-2\lambda_\text{choice})F_L(x;\alpha_L, \beta_L)$$

In [10]:
plt.rcParams['figure.figsize'] = [10, 7]

In [11]:
@interact
def psi_choice(threshold=(-100, 100, 1), slope=(.01, .8, .05), lapse=(0,.1,.01)):
    g=lapse
    x = np.linspace(-100,100,200)
    _, ax = plt.subplots()
    ax.plot(x, 100 * psi(x, lapse, g, F_L, threshold, slope), linewidth=3)
    ax.plot([threshold, threshold], [0, 100*psi(threshold, lapse, g, F_L, threshold, slope)], color='green')
    ax.set(xlim=(-100, 100), ylim=(0,100),
           xlabel="signed coherence",
           ylabel="percent choose 'right'",
           title="Logistic + lapses")

A Jupyter Widget

Illustration of how the two psychometric functions relate:

In [12]:
plt.rcParams['figure.figsize'] = [14, 7]

In [13]:
@interact
def psi_choice_corr(thr_corr=(0.001,100,1),
                    slope_corr=(.01,20,1),
                    thr_choice=(-100, 100, 1), 
                    slope_choice=(.01, .8, .05), 
                    lapse=(0,.1,.01)):
    g = {'corr': 1/2, 'choice': lapse}
    x = {'choice': np.linspace(-100,100, 200), 'corr': np.linspace(0,100,200)}
    F = {'logistic': F_L, 'Weibull': F_W, 'choice': F_L, 'corr': F_W}
    threshold = {'choice': thr_choice, 'corr': thr_corr}
    slope = {'choice': slope_choice, 'corr': slope_corr}
    _, (ax1,ax2) = plt.subplots(1,2)
    ax = {'choice': ax2, 'corr': ax1}
    for case in ('choice', 'corr'):
#         print(case)
        ax[case].plot(x[case], 
                          100 * psi(x[case], lapse, g[case], F[case], 
                                    threshold[case], slope[case]), 
                      linewidth=3, 
                      color='red' if case == 'choice' else 'blue')
        if case == 'corr':
            c = 1/2 * (psi(abs(x[case]), lapse, g['choice'], F['logistic'], threshold['choice'], slope['choice']) + 
                       (1-psi(-abs(x[case]), lapse, g['choice'], F['logistic'], threshold['choice'], slope['choice'])))
#             print(type(c), c.shape)
            ax[case].plot(x[case], 100*c, color='red', linewidth=3)
        lylim = 45 if case == 'corr' else 0
        ax[case].plot([threshold[case], threshold[case]], 
                          [lylim, 100*psi(threshold[case], lapse, g[case], F[case], 
                                       threshold[case], slope[case])], color='green')
        ax[case].set(xlim=(0, 100) if case == 'corr' else (-100,100), 
                     ylim=(lylim, 100),
               xlabel="coherence" if case == 'corr' else "signed coherence",
               ylabel="percent choose 'right'" if case == 'choice' else 'percent correct',
                    title="choose 'right'" if case == 'choice' else 'percent correct')


A Jupyter Widget

## Logistic regression

### Single predictor

In [14]:
plt.rcParams['figure.figsize'] = [14, 7]

The interactive widget below illustrates how the intercept and slope parameters $\beta_0$ and $\beta_1$ in a single predictor logistic regression affect the psychometric function (red). The equation is:
$$logit(P(\text{choose right})):=\log \frac{P(\text{choose right})}{1-P(\text{choose right})}=\beta_0+\beta_1 x$$

In [15]:
@interact
def psi_logit(intercept=(-6, 6, 1), slope=(.01, .8, .05)):
    x = {'choice': np.linspace(-100,100, 200), 'logit': np.linspace(-100,100,200)}
    
    lin_func = lambda c: intercept + slope * c
    y = {'choice': logistic_base(x['choice'], lin_func), 'logit': lin_func(x['logit'])}

    _, (ax1,ax2) = plt.subplots(1,2)
    ax = {'choice': ax2, 'logit': ax1}
    for case in ('choice', 'logit'):
        ax[case].plot(x[case], y[case], 
                      linewidth=3, 
                      color='red' if case == 'choice' else 'blue')
        ylim = (-50,50) if case == 'logit' else (0,1)
        ax[case].set(xlim=(-100, 100), ylim=ylim, xlabel="signed coherence",
                     ylabel="percent choose 'right'" if case == 'choice' else 'log[p/(1-p)]',
                     title="psychometric" if case == 'choice' else 'logit(p)')
        horiz_line = 0 if case == 'logit' else 0.5
        ax[case].hlines(horiz_line, *plt.xlim())
        ax[case].vlines(0, *ylim)

A Jupyter Widget